# Factor de Carga

 - **Desarrollador:** Víctor Echeverría
 - **Fecha:** 2020-09-18
 - **Descripción:** Se realiza la carga de los datos para realizar las ejecuciones calendarizadas de los ETLs de Factor de Carga y carga inicial de datos.

In [1]:
from ExceptionManager import ExceptionManager
from HDFSContext import HDFSContext
from GenericDataFrame import GenericDataFrame
from DBContextDw import DBContextDw
from EtlAL import EtlAL
from LimpiarFactorCarga import LimpiarFactorCarga

from pyspark.sql import functions as Funct
from pyspark.sql.functions import regexp_replace,row_number, col,upper,udf
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from datetime import timedelta
import datetime
import time
from dateutil.relativedelta import relativedelta

In [2]:
dbContext = DBContextDw(Database='dwh_sirio',urlDriver='/home/jovyan/work/postgresql-42.2.12.jar')
accesoDatos = EtlAL(dbContext)
spark = dbContext.spark

def EstablecerPeriodosEjecucion(modulo): 
    """Método que inserta datos de los periodos a ser procesados"""
    try: 

        maxPk = accesoDatos.GetMaxPkTable("select max(tmp_etl_id_pk) pk from cen_cnt.tmp_etl_fecha_carga")
        periodos = []

        tmp_etl_fecha_carga = accesoDatos.GetAllData("cen_cnt.tmp_etl_fecha_carga")\
        .where(col('tmp_etl_modulo') == modulo)

        etl_fecha_carga = accesoDatos.GetAllData("cen_cnt.etl_fecha_carga")\
        .where((col('etl_modulo') == modulo) &\
               (col('etl_forzar') == False)).first() 

        if etl_fecha_carga is not None:
            modulo = etl_fecha_carga.etl_modulo
            fecha_inicio = etl_fecha_carga.etl_fecha_inicio
            fecha_fin_tot = etl_fecha_carga.etl_fecha_fin
            while (fecha_inicio < fecha_fin_tot):
                fecha_fin = (fecha_inicio + relativedelta(years=1)) - timedelta(minutes=1)
                periodos.append([modulo,fecha_inicio, fecha_fin,int(fecha_inicio.strftime('%Y%m%d')),
                                 int(fecha_fin.strftime('%Y%m%d'))])
                fecha_inicio = fecha_fin + timedelta(minutes=1)

        schema = StructType([
            StructField("tmp_etl_modulo", StringType(),False),
            StructField("tmp_etl_fecha_inicio", TimestampType(),False),
            StructField("tmp_etl_fecha_fin", TimestampType(),False),
            StructField("tmp_etl_id_fecha_inicio", LongType(),False),
            StructField("tmp_etl_id_fecha_fin", LongType(),False)])
        df_periodos =  spark.createDataFrame(periodos,schema=schema)
        df_periodos = df_periodos.select((maxPk+row_number()\
                                          .over(Window.partitionBy().orderBy('tmp_etl_fecha_inicio')))\
                                         .alias("tmp_etl_id_pk"),'*')

        salida = accesoDatos.Insert(df_periodos, "cen_cnt.tmp_etl_fecha_carga")

        return salida

    except Exception as error:
        ExceptionManager.Treatment(error)
    
def Procesar(fecha_inicio,fecha_fin,id_fecha_inicio,id_fecha_fin,modulo,deteleTmpQuery):
    salida = LimpiarFactorCarga.Elt_main(fecha_inicio,fecha_fin,modulo,deteleTmpQuery)
    return (id_fecha_inicio,id_fecha_fin,salida)
            
def EjecutarETL(modulo):
    """Método que realiza la carga incremental"""
    try: 
        print("============================= CARGA INCREMENTAL DE DATOS ===================================")
        
        tmp_etl_fecha_carga = accesoDatos.GetAllData('cen_cnt.tmp_etl_fecha_carga')\
        .where(col('tmp_etl_modulo') == modulo)


        if(len(tmp_etl_fecha_carga.head(1))==0):
            print ("\n **** No existen periodos a ejecutar.")
            return None

        fecha_inicio = tmp_etl_fecha_carga.groupby().agg(Funct.min('tmp_etl_fecha_inicio'),
                                                         Funct.min('tmp_etl_id_fecha_inicio'))
        fecha_fin = tmp_etl_fecha_carga.groupby().agg(Funct.max('tmp_etl_fecha_fin'),
                                                      Funct.max('tmp_etl_id_fecha_fin'))

        periodo = "    ### Periodo: Desde => {0}   Hasta => {1}".format(fecha_inicio.first()[0], fecha_fin.first()[0])
        print(str(datetime.datetime.now()))
        print(periodo)
        deteleTmpQuery = 'DELETE FROM cen_cnt.tmp_etl_fecha_carga WHERE tmp_etl_fecha_inicio = \'{0}\' '\
                       'and tmp_etl_fecha_fin = \'{1}\' and tmp_etl_modulo = \'{2}\' '

        deteleFactQuery = 'DELETE FROM cen_dws.fact_factor_carga WHERE tmpo_id_fk between {0} and {1}'\
        .format(fecha_inicio.first()[1],fecha_fin.first()[1])


        transactDeleteFact = accesoDatos.Delete(deteleFactQuery)
        
        if transactDeleteFact == True: 
            procesamiento = tmp_etl_fecha_carga.rdd.map(lambda x: Procesar(x.tmp_etl_fecha_inicio,
                                                                           x.tmp_etl_fecha_fin,
                                                                           x.tmp_etl_id_fecha_inicio,
                                                                           x.tmp_etl_id_fecha_fin,
                                                                           modulo,
                                                                           deteleTmpQuery))
            for d in procesamiento.collect():
                print(d)
        print ("\n **** EXITOSO: Se ha procesado todos los periodos del módulo {0}".format(modulo))
        print(str(datetime.datetime.now()))
    except Exception as error:
        ExceptionManager.Treatment(error)

In [3]:
#EstablecerPeriodosEjecucion('FACTOR DE CARGA')
EjecutarETL('FACTOR DE CARGA')

============================= CARGA INCREMENTAL DE DATOS ===================================
2020-09-18 20:35:27.521095
    ### Periodo: Desde => 2016-01-01 00:00:00   Hasta => 2020-12-31 23:59:00
(20160101, 20161231, True)
(20170101, 20171231, True)
(20180101, 20181231, True)
(20190101, 20191231, True)
(20200101, 20201231, True)

 **** EXITOSO: Se ha procesado todos los periodos del módulo FACTOR DE CARGA
2020-09-18 20:37:58.177978
